In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensornetwork as tn
%matplotlib inline

plt.rcParams['figure.figsize'] = [9, 5]

# Problem 1: Tensor contractions

In [ ]:
dim = 20
A = np.random.random([dim]*4)
B = np.random.random([dim]*4)
A.shape, B.shape

Let's contract the tensors A and B using a fast BLAS matrix multiplication
(Check ``np.show_config()`` that you have BLAS for speed ups).

The contraction we consider equals the sum
$$ \sum_{mn} A_{imjn} B_{mkln}.$$


In [ ]:
# declaring the tensors as nodes
An = tn.Node(A, name='A')
Bn = tn.Node(B, name='B')
# connecting the nodes
tn.connect(An[1], Bn[0])
tn.connect(An[3], Bn[3])
tn.to_graphviz([An, Bn])

In [ ]:
# contraction consumes the original network, therefor we work on a copy
ABn = tn.contract_between(*tn.replicate_nodes([An, Bn]))
AB_einsum = np.einsum('imjn,mkln', A, B)
assert np.allclose(ABn.tensor, AB_einsum)

In [ ]:
# We can also name the nodes
# declaring the tensors as axis
An = tn.Node(A, name='A', axis_names="imjn")
Bn = tn.Node(B, name='B', axis_names="mkln")
# connecting the nodes
tn.connect(An["m"], Bn["m"], name="m")
tn.connect(An["n"], Bn["n"], name="n")
tn.to_graphviz([An, Bn])

In [ ]:
# Lets also check speed
%timeit tn.contract_between(*tn.replicate_nodes([An, Bn]))

In [ ]:
# We can speed things up using "jax",
# it can even be used to put things on the GPU
tn.set_default_backend("jax")
# We can also name the nodes
# declaring the tensors as axis
An = tn.Node(A, name='A', axis_names="imjn")
Bn = tn.Node(B, name='B', axis_names="mkln")
# connecting the nodes
tn.connect(An["m"], Bn["m"], name="m")
tn.connect(An["n"], Bn["n"], name="n")
%timeit tn.contract_between(*tn.replicate_nodes([An, Bn]))

# Problem 2: Compression

In [ ]:
from scipy import misc
face = misc.face(gray=True)
plt.imshow(face, cmap='gray');

In [ ]:
facen = tn.Node(face.astype(float), name='face')

# truncate the singular values for compression
num = 150
us, vhs, truncs = tn.split_node(facen, max_singular_values=num,
                                left_edges=[facen[0]], right_edges=[facen[1]])
face_trunc = tn.contract_between(us, vhs)
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face, cmap="gray")
axes[1].set_title("compressed")
axes[1].imshow(face_trunc.tensor, cmap="gray")
print("Compression:", (us.tensor.size + vhs.tensor.size)/face.size)

In [ ]:
# the error is given by the truncated singular values
# Frobenius norm
ferr_singular = np.sqrt(np.sum(truncs**2))
ferr_norm = np.linalg.norm(face - face_trunc.tensor, ord='fro')
print("Frobenius norm:", ferr_singular, ferr_norm)
# Spectral norm
serr_singular = truncs[0]
serr_norm = np.linalg.norm(face - face_trunc.tensor, ord=2)
print("Spectral norm:", serr_singular, serr_norm)

In [ ]:
# alternatively, we can directly limit the truncation error:
facen = tn.Node(face.astype(float), name='face')

# truncate the singular values for compression
error = 1e-1
us, vhs, truncs = tn.split_node(facen, max_truncation_err=error, relative=True,
                                left_edges=[facen[0]], right_edges=[facen[1]])
print(f"Truncated {truncs.size} values, that is {min(face.shape) - truncs.size} are kept")
face_trunc = tn.contract_between(us, vhs)
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face, cmap="gray")
axes[1].set_title("compressed")
axes[1].imshow(face_trunc.tensor, cmap="gray");
print("Compression:", (us.tensor.size + vhs.tensor.size)/face.size)
# the error is given by the truncated singular values
# Frobenius norm
ferr_singular = np.sqrt(np.sum(truncs**2))
ferr_norm = np.linalg.norm(face - face_trunc.tensor, ord='fro')
print("Frobenius norm:", ferr_singular, ferr_norm,
      "\trelative", ferr_norm / np.linalg.norm(face, ord="fro"))
# Spectral norm
serr_singular = truncs[0]
serr_norm = np.linalg.norm(face - face_trunc.tensor, ord=2)
print("Spectral norm: ", serr_singular, serr_norm,
      "\trelative", serr_norm / np.linalg.norm(face, ord=2))

Let's also compress the colored image, spliting the tensor between rows and columns of the picture:

In [ ]:
face = misc.face(gray=False)
plt.imshow(face);

In [ ]:
facen = tn.Node(face.astype(float), name='face')

# truncate the singular values for compression
num = 150
us, vhs, truncs = tn.split_node(facen, max_singular_values=num,
                                left_edges=facen[:1], right_edges=facen[1:])
tn.to_graphviz([us, vhs])

In [ ]:
face_trunc = tn.contract_between(us, vhs)
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face)
axes[1].set_title("compressed")
axes[1].imshow(face_trunc.tensor.astype(int));
print("Compression:", (us.tensor.size + vhs.tensor.size)/face.size)

Of course, we can further split of the color dimension.
In general the quality of approximation/compression will depend on the geometry we use:

In [ ]:
facen = tn.Node(face.astype(float), name='face')
# truncate the singular values for compression
err = 1e-1 
row, vhs, truncs = tn.split_node(facen, max_truncation_err=error/2, relative=True,
                                 left_edges=facen[:1], right_edges=facen[1:],
                                 left_name='row')                            
print(f"(1) Truncated {truncs.size} values")

column, color, truncs2 = tn.split_node(vhs, max_truncation_err=error/2, relative=True,
                                       left_edges=vhs[:2], right_edges=vhs[2:],
                                       left_name='column', right_name='color')
print(f"(2) Truncated {truncs2.size} values")
tn.to_graphviz([row, column, color])

In [ ]:
# work with copy in case of replicated execusion of cell
tensors = tn.replicate_nodes([row, column, color])
face_trunc = tn.contractors.auto(tensors,
                                 output_edge_order=[tensors[0][0], tensors[1][1], tensors[2][1]])
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face)
axes[1].set_title("compressed")
axes[1].imshow(face_trunc.tensor.astype(int));
print("Compression:", sum(node.tensor.size for node in tensors)/face.size)
ferr_singular = np.sqrt(np.sum(truncs**2)) + np.sqrt(np.sum(truncs2**2))

ferr_norm = np.linalg.norm((face - face_trunc.tensor).reshape(-1))
print("Frobenius norm:", ferr_singular, ferr_norm,
      "\trelative", ferr_norm / np.linalg.norm(face.reshape(-1)))


Spliting the colors of in a second step doesn't allow for any additional compression,
there can only be 3 singular values and it is intuitive that we need all to reasonably represent colors.
However we can instead but the color-dimension into the center of the tensor train:


In [ ]:
facen = tn.Node(face.astype(float), name='face')
# truncate the singular values for compression
num = 150
err = 1e-1 
row, vhs, truncs = tn.split_node(facen, max_truncation_err=error/2, relative=True,
                                 left_edges=facen[:1], right_edges=facen[1:],
                                 left_name='row')
print(f"(1) Truncated {truncs.size} values")
color, column, truncs2 = tn.split_node(vhs, max_truncation_err=error/2, relative=True,

                                       left_edges=[vhs[0], vhs[2]], right_edges=[vhs[1]],
                                       left_name='color', right_name='column')
print(f"(2) Truncated {truncs2.size} values")
tn.to_graphviz([row, column, color])

In [ ]:
# work with copy in case of replicated execusion of cell
tensors = tn.replicate_nodes([row, column, color])
face_trunc = tn.contractors.auto(tensors,
                                 output_edge_order=[tensors[0][0], tensors[1][1], tensors[2][1]])
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face)
axes[1].set_title("compressed")
axes[1].imshow(face_trunc.tensor.astype(int));
print("Compression:", sum(node.tensor.size for node in tensors)/face.size)

ferr_singular = np.sqrt(np.sum(truncs**2)) + np.sqrt(np.sum(truncs2**2))

ferr_norm = np.linalg.norm((face - face_trunc.tensor).reshape(-1))
print("Frobenius norm:", ferr_singular, ferr_norm,
      "\trelative", ferr_norm / np.linalg.norm(face.reshape(-1)))